Import Database

In [4]:
import pandas as pd
df = pd.read_csv('uoftdb.csv')
data = {'id':[], 'day':[], 'pos':[],'signal':[]}
for elem in df:
    params = elem.split('_')
    data['id'].append(params[0])
    data['day'].append(params[1])
    data['pos'].append(params[2])
    data['signal'].append(df[elem].to_list())
df_new = pd.DataFrame.from_dict(data)
# df_new.head()
df_sitted = df_new[df_new['pos'] == "sit"]

,id,day,pos,signal
0,1,1,sit,"[0.994, 1.008, 1.235, 1.06, 0.985, 1.168, 1.14..."
1,2,1,sit,"[1.207, 1.328, 1.157, 1.159, 1.308, 1.182, 1.1..."
2,3,1,sit,"[0.937, 0.735, 1.293, 0.91, 0.588, 1.066, 1.12..."
3,4,1,sit,"[0.823, 0.712, 1.841, 1.133, 0.609, 1.436, 1.5..."
4,5,1,sit,"[1.078, 1.018, 1.053, 1.095, 1.036, 1.027, 1.0..."


Filtering

In [33]:
from scipy.signal import butter,filtfilt
import plotly.graph_objects as go
# Filter requirements.
T = 5.0         # Sample Period
fs = 200.0       # sample rate, Hz
cutoff = [0.5, 40]      # desired cutoff frequency of the filter, Hz ,      slightly higher than actual 1.2 Hz
order = 6       
n = int(T * fs) # total number of samples
data = df['1_1_sit'].to_numpy()
data = data[4000:5000]
def butter_bandpass_filter(data, cutoff, fs, order):
    nyq = fs/2
    normal_cutoff = [f/nyq for f in cutoff]
    # Get the filter coefficients 
    b, a = butter(order, normal_cutoff, btype='bandpass', analog=False)
    y = filtfilt(b, a, data)
    return y

filttered = butter_bandpass_filter(data, cutoff, fs, order)
fig = go.Figure()
fig.add_trace(go.Scatter(
            y = data,
            line =  dict(shape =  'spline' ),
            name = 'signal with noise'
            ))
fig.add_trace(go.Scatter(
            y = filttered,
            line =  dict(shape =  'spline' ),
            name = 'filtered signal'
            ))
fig.show()